Took the nearest neighbors of the L-KNN bars and applied them to the last 4 for kernel estimates 
These neighbors are filtered out because their lookahead target price (for labeling the neighbor’s movement) is not available.

Only neighbors that have a valid lookahead target within the time series remain and contribute votes.

For the very last bars, this reduces the effective number of neighbors used for classification — sometimes from K_NEIGHBORS (e.g., 8) down to fewer neighbors (here, 5).



In [ ]:
import nest_asyncio
nest_asyncio.apply()
import asyncio
from ib_insync import *
import pandas as pd
import numpy as np
import ta
import plotly.graph_objects as go

"""
Filter parameters
-----------------
VOL_WINDOW:       ATR period (bars) for volatility calculation (consider higher for daily)
VOL_THRESHOLD:    Minimum ATR percent (ATR/close) for a bar to be considered volatile enough for trading (e.g., 0.01 = 1%)
REGIME_THRESHOLD: Rolling mean of returns must be above this (e.g., -0.1) to be considered "trending" regime
ADX_PERIOD:       ADX calculation period (bars)
ADX_THRESHOLD:    Minimum ADX value to confirm market is trending (20 = classic)
SMA_PERIOD:       SMA bars for trend filter (only long if above, short if below)
"""

# ========= CHART SETTINGS =========
# For intraday bars (e.g., 1-minute), use:
#BAR_SIZE = '1 min'
#DURATION = '14 D'
#NUM_BARS = 2000

# For daily chart (uncomment the three lines below for daily bars):
BAR_SIZE = '1 day'      # IBKR bar size for daily candles
DURATION = '5 Y'        # How far back to get data (e.g., '5 Y' = 5 years)
NUM_BARS = 2000         # Approximate daily bars in 5 years
# ================================

# Filter tuning (adjust to your needs)
VOL_WINDOW       = 30        # ATR lookback period (bars); for daily, 14-30 is common
VOL_THRESHOLD    = 0.001     # Minimum ATR% for volatility filter (e.g., 0.01=1%); 0.006-0.01 for dailies
REGIME_THRESHOLD = -0.1      # Regime filter: rolling mean of returns must be greater than this to signal a trend
ADX_PERIOD       = 14        # ADX lookback window (bars)
ADX_THRESHOLD    = 20        # Minimum ADX to allow signals (trend filter)
SMA_PERIOD       = 200       # SMA filter period (bars)

SYMBOL      = 'AAL'
EXCHANGE    = 'SMART'
CURRENCY    = 'USD'
LOOKAHEAD   = 4
K_NEIGHBORS = 8

# === Add or remove features here as desired ===
# You can add 'MACD', 'MACD_signal', or 'MACD_hist' (histogram) as features
FEATURES = ['RSI', 'ADX', 'CCI', 'MACD']

# ==== Data Fetch Function (works with any BAR_SIZE or DURATION) ====
async def fetch_ibkr_stock(symbol, exchange, currency, bar_size, duration, num_bars):
    """Fetches OHLCV bars from IBKR using specified bar size/duration."""
    ib = IB()
    await ib.connectAsync('127.0.0.1', 7497, clientId=np.random.randint(1000, 10000))
    contract = Stock(symbol, exchange, currency)
    await ib.qualifyContractsAsync(contract)
    bars = await ib.reqHistoricalDataAsync(
        contract=contract,
        endDateTime='',
        durationStr=duration,
        barSizeSetting=bar_size,
        whatToShow='TRADES',
        useRTH=True,        # Only main session hours
        formatDate=1,
        keepUpToDate=False
    )
    df = util.df(bars)
    df.set_index('date', inplace=True)
    df = df.tail(num_bars)
    ib.disconnect()
    return df

# ==== ML Functions and Filters ====

def lorentzian_distance(a, b):
    """Lorentzian distance metric for nearest neighbor calculations."""
    return np.sum(np.log(1 + np.abs(a - b)))

def volatility_filter(df, vol_window, vol_thresh):
    """ATR percent-based volatility filter. Returns Boolean mask."""
    atr = ta.volatility.AverageTrueRange(df['high'], df['low'], df['close'], window=vol_window).average_true_range()
    atr_pct = atr / df['close']
    return atr_pct > vol_thresh

def regime_filter(df, threshold):
    """Regime filter: rolling mean of returns must exceed threshold."""
    close_returns = df['close'].pct_change()
    regime = close_returns.rolling(window=20).mean()
    return regime > threshold

def adx_filter(df, adx_period, adx_threshold):
    """ADX trend filter: ADX must exceed threshold."""
    adx = ta.trend.ADXIndicator(df['high'], df['low'], df['close'], window=adx_period).adx()
    return adx > adx_threshold

def classify_lorentzian_knn_with_filters(
        df, features, sma_period,
        vol_window, vol_thresh,
        regime_thresh, adx_period, adx_thresh,
        n_neighbors=8, lookahead=4, max_bars_back=500):
    """
    L-KNN prediction with regime, volatility, ADX, and SMA filters.
    Trades only allowed if all filters pass.
    Explicit handling for last `lookahead` bars with neighbor filtering.
    Returns predictions and neighbor votes info for each prediction point.
    """
    pred = np.full(len(df), np.nan)
    neighbors_info = {}

    closes = df['close'].values
    feature_mat = df[features].values
    sma = df['SMA'].values

    vol_filter = volatility_filter(df, vol_window, vol_thresh)
    regime_filt = regime_filter(df, regime_thresh)
    adx_filt = adx_filter(df, adx_period, adx_thresh)

    length = len(df)

    # Main loop - exclude last 'lookahead' bars
    for idx in range(max_bars_back, length - lookahead):
        if np.isnan(sma[idx]) or closes[idx] == 0:
            continue
        if not (vol_filter.iloc[idx] and regime_filt.iloc[idx] and adx_filt.iloc[idx]):
            continue

        anchor_start = max(0, idx - max_bars_back)
        anchor_indices = np.arange(anchor_start, idx)
        anchor_feats = feature_mat[anchor_indices, :]
        target = feature_mat[idx]

        dists = np.array([lorentzian_distance(target, anchor_feats[j]) for j in range(anchor_feats.shape[0])])
        if len(dists) < n_neighbors:
            continue

        knn_indices = dists.argsort()[:n_neighbors]
        valid = anchor_indices[knn_indices] + lookahead < length  # neighbors with valid lookahead
        selected = anchor_indices[knn_indices][valid]
        if len(selected) == 0:
            continue

        y_train = (closes[selected + lookahead] > closes[selected]).astype(int) - \
                  (closes[selected + lookahead] < closes[selected]).astype(int)
        vote = y_train.sum()
        raw_signal = np.sign(vote) if vote != 0 else 0

        if raw_signal == 1 and closes[idx] > sma[idx]:
            pred[idx] = 1
        elif raw_signal == -1 and closes[idx] < sma[idx]:
            pred[idx] = -1
        else:
            pred[idx] = 0

        neighbors_info[idx] = y_train.tolist()

    # Explicit classification for last 'lookahead' bars
    for idx in range(length - lookahead, length):
        if np.isnan(sma[idx]) or closes[idx] == 0:
            continue
        if not (vol_filter.iloc[idx] and regime_filt.iloc[idx] and adx_filt.iloc[idx]):
            continue

        anchor_start = max(0, idx - max_bars_back)
        anchor_indices = np.arange(anchor_start, idx)
        anchor_feats = feature_mat[anchor_indices, :]
        target = feature_mat[idx]

        dists = np.array([lorentzian_distance(target, anchor_feats[j]) for j in range(anchor_feats.shape[0])])
        if len(dists) < n_neighbors:
            continue

        knn_indices = dists.argsort()[:n_neighbors]
        # For last bars, explicitly filter neighbors with valid lookahead
        valid = anchor_indices[knn_indices] + lookahead < length
        selected = anchor_indices[knn_indices][valid]
        if len(selected) == 0:
            continue

        y_train = (closes[selected + lookahead] > closes[selected]).astype(int) - \
                  (closes[selected + lookahead] < closes[selected]).astype(int)
        vote = y_train.sum()
        raw_signal = np.sign(vote) if vote != 0 else 0

        if raw_signal == 1 and closes[idx] > sma[idx]:
            pred[idx] = 1
        elif raw_signal == -1 and closes[idx] < sma[idx]:
            pred[idx] = -1
        else:
            pred[idx] = 0

        neighbors_info[idx] = y_train.tolist()

    return pred, neighbors_info

# ==== Main Pipeline ====
async def main():
    df = await fetch_ibkr_stock(SYMBOL, EXCHANGE, CURRENCY, BAR_SIZE, DURATION, NUM_BARS)

    # Feature Engineering
    df['RSI'] = ta.momentum.RSIIndicator(df['close'], window=14).rsi()
    df['ADX'] = ta.trend.ADXIndicator(df['high'], df['low'], df['close'], window=14).adx()
    df['CCI'] = ta.trend.CCIIndicator(df['high'], df['low'], df['close'], window=20).cci()
    df['SMA'] = ta.trend.SMAIndicator(df['close'], window=SMA_PERIOD).sma_indicator()
    macd_ind = ta.trend.MACD(df['close'], window_slow=26, window_fast=12, window_sign=9)
    df['MACD']        = macd_ind.macd()
    df['MACD_signal'] = macd_ind.macd_signal()
    df['MACD_hist']   = macd_ind.macd_diff()
    df = df.dropna().reset_index()

    preds, neighbors_info = classify_lorentzian_knn_with_filters(
        df, FEATURES, SMA_PERIOD, VOL_WINDOW, VOL_THRESHOLD,
        REGIME_THRESHOLD, ADX_PERIOD, ADX_THRESHOLD,
        n_neighbors=K_NEIGHBORS, lookahead=LOOKAHEAD, max_bars_back=200
    )

    df['raw_pred'] = preds

    # Trade state logic: trade only on state change (entry flip)
    df['trade_signal'] = np.nan
    prev_position = 0
    for i, row in df.iterrows():
        sig = row['raw_pred']
        if np.isnan(sig) or sig == 0:
            continue
        sig = int(sig)
        if sig != prev_position:
            df.at[i, 'trade_signal'] = sig
            prev_position = sig

    # Interactive Chart
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df['date'], y=df['close'],
        mode='lines', name='Close Price', line=dict(color='black', width=2)
    ))
    fig.add_trace(go.Scatter(
        x=df.loc[df['raw_pred'] == 1, 'date'], y=df.loc[df['raw_pred'] == 1, 'close'],
        mode='markers', name='Kernel Long', marker=dict(color='green', size=8, symbol='circle')
    ))
    fig.add_trace(go.Scatter(
        x=df.loc[df['raw_pred'] == -1, 'date'], y=df.loc[df['raw_pred'] == -1, 'close'],
        mode='markers', name='Kernel Short', marker=dict(color='red', size=8, symbol='circle')
    ))
    buys = df[df['trade_signal'] == 1]
    sells = df[df['trade_signal'] == -1]
    fig.add_trace(go.Scatter(
        x=buys['date'], y=buys['close'],
        mode='markers', marker=dict(symbol='triangle-up', color='green', size=15, line=dict(color='black', width=2)),
        name='Buy Signal'
    ))
    fig.add_trace(go.Scatter(
        x=sells['date'], y=sells['close'],
        mode='markers', marker=dict(symbol='triangle-down', color='red', size=15, line=dict(color='black', width=2)),
        name='Sell Signal'
    ))
    fig.update_layout(
        title=f"{SYMBOL} | Lorentzian KNN with MACD, Vol, Regime, ADX, SMA Filters",
        xaxis_title="Date/Time",
        yaxis_title="Price",
        hovermode='x unified',
        template='plotly_white',
        width=1100, height=650,
        legend=dict(x=0.01, y=0.99)
    )
    fig.show()

    # Print last 10 predictions with neighbor votes
    prediction_indices = [i for i in range(len(df)) if not np.isnan(preds[i])]
    last_10_idxs = prediction_indices[-10:]
    print("Last 10 Predictions and their neighbor votes:")
    for idx in last_10_idxs:
        date = df.loc[idx, 'date']
        pred_val = preds[idx]
        neighbor_votes = neighbors_info.get(idx, [])
        print(f"Date: {date}, Prediction: {pred_val}, Neighbor votes: {neighbor_votes}")

    return df

# --- Run the script ---
df_signals = asyncio.run(main())
df = df_signals


Last 10 Predictions and their neighbor votes:
Date: 2025-06-20, Prediction: 0.0, Neighbor votes: [1, 1, -1, -1, -1, 1, 1, 1]
Date: 2025-06-23, Prediction: 0.0, Neighbor votes: [1, 1, 1, -1, -1, 1, -1, -1]
Date: 2025-06-24, Prediction: 0.0, Neighbor votes: [1, 1, -1, -1, -1, -1, 1, 1]
Date: 2025-07-16, Prediction: 0.0, Neighbor votes: [-1, -1, 1, -1, 1, -1, 1, -1]
Date: 2025-07-17, Prediction: 0.0, Neighbor votes: [1, 1, -1, -1, -1, -1, -1, 1]
Date: 2025-07-18, Prediction: 0.0, Neighbor votes: [-1, 1, 1, -1, -1, 1, -1, -1]
Date: 2025-07-21, Prediction: 0.0, Neighbor votes: [-1, -1, -1, -1, -1, 1, -1, -1]
Date: 2025-07-22, Prediction: 0.0, Neighbor votes: [-1, 1, -1, -1, -1, -1, -1]
Date: 2025-07-23, Prediction: 0.0, Neighbor votes: [-1, -1, -1, 1, -1, -1]
Date: 2025-07-24, Prediction: 0.0, Neighbor votes: [-1, -1, -1, -1, -1]


In [ ]:
# --- EVALUATE PREDICTION ACCURACY AND RETURNS OVER 4-BAR HORIZON ---

# Calculate 4-bar forward returns
df['forward_return'] = df['close'].shift(-LOOKAHEAD) / df['close'] - 1

# Filter to bars where a prediction was made and the future is available
mask = df['raw_pred'].notna() & (df['raw_pred'] != 0) & df['forward_return'].notna()

# Evaluate correct direction
correct = np.where(
    (df['raw_pred'] == 1) & (df['forward_return'] > 0), 1,
    np.where((df['raw_pred'] == -1) & (df['forward_return'] < 0), 1, 0)
)
# Only keep for valid prediction bars
correct = correct[mask.values]

# Long/short statistics
long_mask = mask & (df['raw_pred'] == 1)
short_mask = mask & (df['raw_pred'] == -1)

long_returns = df.loc[long_mask, 'forward_return']
short_returns = df.loc[short_mask, 'forward_return']

# Display results
print('\n---- STRATEGY PERFORMANCE METRICS OVER 4-BAR HORIZON ----')
print(f'Number of Predictions Scored: {mask.sum()}')
print(f'Percent Correct: {100 * correct.mean():.2f}%')
print(f'Long Signals:   {long_mask.sum()}  | Avg Return: {100 * long_returns.mean():.3f}%')
print(f'Short Signals:  {short_mask.sum()}  | Avg Return: {100 * short_returns.mean():.3f}%')
print('--------------------------------------------------------\n')

# Optionally, display a Markdown table of results
import pandas as pd
summary = pd.DataFrame({
    'Num Signals': [long_mask.sum(), short_mask.sum()],
    'Avg 4-Bar Ret %': [long_returns.mean() * 100, short_returns.mean() * 100]
}, index=['Long', 'Short'])
display(summary)



---- STRATEGY PERFORMANCE METRICS OVER 4-BAR HORIZON ----
Number of Predictions Scored: 239
Percent Correct: 64.44%
Long Signals:   186  | Avg Return: 0.739%
Short Signals:  53  | Avg Return: -2.127%
--------------------------------------------------------



,Num Signals,Avg 4-Bar Ret %
Long,186,0.738958
Short,53,-2.126964
